In [1]:
import pandas as pd
import numpy as np
import os
import gc

PATH = '/home/kai/data/kaggle/homecredit/'

train = pd.read_csv(PATH + 'application_train.csv')
test = pd.read_csv(PATH + 'application_test.csv')
'done'

'done'

In [2]:
df = pd.concat([train,test], axis = 0, sort = False)

# Factorize

In [4]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / df[col2]

def multiply_name(col1, col2): return col1 + '_times_' + col2
def multiply(df, col1, col2): return df[col1] * df[col2]

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col, dummy_na = True)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [ ]:
flag_other = [col for col in train.columns.values if 'FLAG' in col and 'DOCUMENT' not in col]
df['CONTACT'] = df[['FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE','FLAG_EMAIL']].sum(axis = 1)

df.drop(['FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE'], axis = 1, inplace = True)

df['FLAG_OTHER'] = df[['FLAG_OWN_CAR',
 'FLAG_OWN_REALTY']].sum(axis = 1)

In [15]:
set(df.EMERGENCYSTATE_MODE)

{'No', 'Yes', nan}

## Factorize

In [ ]:
categorical_columns = [col for col in df.columns if df[col].dtype == 'object'] + ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

factor_columns = [col for col in categorical_columns if len(set(df[col])) == 2] + ['CODE_GENDER', 'WEEKDAY_APPR_PROCESS_START']

categorical_columns = [i for i in categorical_columns if i not in factor_columns]


categorical_columns = [col for col in df.columns if df[col].dtype == 'object']

for bin_feature in factor_columns:
    print(bin_feature)
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

# AREA

In [6]:
x = df[l]
x.fillna(0 , inplace = True)
df[l] = x

/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3787: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [7]:
area = ['APARTMENTS','BASEMENTAREA', 'COMMONAREA','LANDAREA', 'LIVINGAPARTMENTS','LIVINGAREA',\
           'NONLIVINGAPARTMENTS', 'NONLIVINGAREA']

for x in ['_AVG', '_MODE', '_MEDI']:
    tmp = [y + x for y in area]
    print(tmp)
    df['livingsum' + x] = df[tmp].sum(axis=1)

['APARTMENTS_AVG', 'BASEMENTAREA_AVG', 'COMMONAREA_AVG', 'LANDAREA_AVG', 'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG', 'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG']
['APARTMENTS_MODE', 'BASEMENTAREA_MODE', 'COMMONAREA_MODE', 'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE', 'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE']
['APARTMENTS_MEDI', 'BASEMENTAREA_MEDI', 'COMMONAREA_MEDI', 'LANDAREA_MEDI', 'LIVINGAPARTMENTS_MEDI', 'LIVINGAREA_MEDI', 'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAREA_MEDI']


In [8]:
df.shape

(356255, 125)

# 1. Do the ratio wrt to count of family members & living area

In [10]:
# 1. Ratio of count of family members(for members less than 4) and living area

In [11]:
members_series = df.CNT_FAM_MEMBERS < 4

area_list = ['livingsum_AVG','livingsum_MODE','livingsum_MEDI']

In [12]:
df['members'] = np.nan

df['members'].loc[members_series] = df['CNT_FAM_MEMBERS'].loc[members_series]

for col in area_list:
    df[ratio_name(col, 'members')] = ratio(df, col, 'members')
    print(ratio_name(col, 'members'))
    
df.drop('members', inplace = True,axis = 1)

/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


livingsum_AVG_divide_members
livingsum_MODE_divide_members
livingsum_MEDI_divide_members


In [13]:
df.shape

(356255, 128)

In [14]:
df['livingsum_AVG_divide_members'].head()

0    0.1520
1    0.1841
2    0.0000
3    0.0000
4    0.0000
Name: livingsum_AVG_divide_members, dtype: float64

# 2. Multiply area rating & living area

In [15]:
# def multiply_name(col1, col2): return col1 + '_times_' + col2
# def multiply(df, col1, col2): return df[col1] * df[col2]

In [16]:
rating = ['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY']
area_list = ['livingsum_AVG','livingsum_MODE','livingsum_MEDI']
for i in rating:
    for j in area_list:
        df[multiply_name(i, j)] = multiply(df, i, i)
        print(multiply_name(i, j))

REGION_RATING_CLIENT_times_livingsum_AVG
REGION_RATING_CLIENT_times_livingsum_MODE
REGION_RATING_CLIENT_times_livingsum_MEDI
REGION_RATING_CLIENT_W_CITY_times_livingsum_AVG
REGION_RATING_CLIENT_W_CITY_times_livingsum_MODE
REGION_RATING_CLIENT_W_CITY_times_livingsum_MEDI


In [17]:
df.shape

(356255, 134)

# Do the ratio wrt to amt-dpd and region-population

In [18]:
obs = ['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']
pop = ['REGION_POPULATION_RELATIVE']

In [19]:
for col in obs:
    df[ratio_name(col, 'REGION_POPULATION_RELATIVE')] = ratio(df, col, 'REGION_POPULATION_RELATIVE')
    print(ratio_name(col, 'REGION_POPULATION_RELATIVE'))

OBS_30_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
DEF_30_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
OBS_60_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
DEF_60_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE


In [20]:
df[['OBS_30_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE']]

,OBS_30_CNT_SOCIAL_CIRCLE_divide_REGION_POPULATION_RELATIVE
0,106.377320
1,282.406100
2,0.000000
3,249.407657
4,0.000000
5,0.000000
6,27.939204
7,640.614990
8,53.665343
9,101.579562


In [21]:
df.shape

(356255, 138)

# DEF/OBS & 60/30

In [22]:
obs_30 = ['OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE']
obs_60 = ['OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']

In [23]:
for i in obs_60:
    for j in obs_30:
        print(ratio_name(i,j))
        df[ratio_name(i,j)] = ratio(df, i, j)

OBS_60_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
OBS_60_CNT_SOCIAL_CIRCLE_divide_DEF_30_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
DEF_60_CNT_SOCIAL_CIRCLE_divide_DEF_30_CNT_SOCIAL_CIRCLE


In [24]:
df.shape

(356255, 142)

In [25]:
default = ['DEF_30_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE']
obs = ['OBS_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE']

In [26]:
for i in default:
    for j in obs:
        print(ratio_name(i,j))
        df[ratio_name(i,j)] = ratio(df, i, j)
        print(df.shape)

DEF_30_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
(356255, 143)
DEF_30_CNT_SOCIAL_CIRCLE_divide_OBS_60_CNT_SOCIAL_CIRCLE
(356255, 144)
DEF_60_CNT_SOCIAL_CIRCLE_divide_OBS_30_CNT_SOCIAL_CIRCLE
(356255, 144)
DEF_60_CNT_SOCIAL_CIRCLE_divide_OBS_60_CNT_SOCIAL_CIRCLE
(356255, 145)


# Other Ratios:

In [36]:
can_be_numerator = [x for x in train.columns if 'AMT' in x] + ['OWN_CAR_AGE', 'DAYS_LAST_PHONE_CHANGE']
can_be_denominator = [x for x in train.columns if 'CNT' in x or 'DAYS' in x] + can_be_numerator

print(len(can_be_numerator),can_be_numerator)
print(len(can_be_denominator),can_be_denominator)

12 ['AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'OWN_CAR_AGE', 'DAYS_LAST_PHONE_CHANGE']
23 ['CNT_CHILDREN', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'CNT_FAM_MEMBERS', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'OWN_CAR_AGE', 'DAYS_LAST_PHONE_CHANGE']


In [37]:
i = 0
for x in can_be_numerator:
    for y in can_be_denominator:
        if y != x and ratio_name(y,x) not in df.columns:
            i += 1
            print(ratio_name(x,y))
            df[ratio_name(x, y)] = ratio(df, x, y)
print(i, df.shape)

AMT_INCOME_TOTAL_divide_CNT_CHILDREN
AMT_INCOME_TOTAL_divide_DAYS_BIRTH
AMT_INCOME_TOTAL_divide_DAYS_EMPLOYED
AMT_INCOME_TOTAL_divide_DAYS_REGISTRATION
AMT_INCOME_TOTAL_divide_DAYS_ID_PUBLISH
AMT_INCOME_TOTAL_divide_CNT_FAM_MEMBERS
AMT_INCOME_TOTAL_divide_OBS_30_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_DEF_30_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_OBS_60_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_DEF_60_CNT_SOCIAL_CIRCLE
AMT_INCOME_TOTAL_divide_DAYS_LAST_PHONE_CHANGE
AMT_INCOME_TOTAL_divide_AMT_CREDIT
AMT_INCOME_TOTAL_divide_AMT_ANNUITY
AMT_INCOME_TOTAL_divide_AMT_GOODS_PRICE
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_HOUR
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_DAY
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_WEEK
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_MON
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_QRT
AMT_INCOME_TOTAL_divide_AMT_REQ_CREDIT_BUREAU_YEAR
AMT_INCOME_TOTAL_divide_OWN_CAR_AGE
AMT_INCOME_TOTAL_divide_DAYS_LAST_PHONE_CHANGE
AMT_CREDIT_divide_CNT_CHILDR

OWN_CAR_AGE_divide_DAYS_ID_PUBLISH
OWN_CAR_AGE_divide_CNT_FAM_MEMBERS
OWN_CAR_AGE_divide_OBS_30_CNT_SOCIAL_CIRCLE
OWN_CAR_AGE_divide_DEF_30_CNT_SOCIAL_CIRCLE
OWN_CAR_AGE_divide_OBS_60_CNT_SOCIAL_CIRCLE
OWN_CAR_AGE_divide_DEF_60_CNT_SOCIAL_CIRCLE
OWN_CAR_AGE_divide_DAYS_LAST_PHONE_CHANGE
OWN_CAR_AGE_divide_DAYS_LAST_PHONE_CHANGE
DAYS_LAST_PHONE_CHANGE_divide_CNT_CHILDREN
DAYS_LAST_PHONE_CHANGE_divide_DAYS_BIRTH
DAYS_LAST_PHONE_CHANGE_divide_DAYS_EMPLOYED
DAYS_LAST_PHONE_CHANGE_divide_DAYS_REGISTRATION
DAYS_LAST_PHONE_CHANGE_divide_DAYS_ID_PUBLISH
DAYS_LAST_PHONE_CHANGE_divide_CNT_FAM_MEMBERS
DAYS_LAST_PHONE_CHANGE_divide_OBS_30_CNT_SOCIAL_CIRCLE
DAYS_LAST_PHONE_CHANGE_divide_DEF_30_CNT_SOCIAL_CIRCLE
DAYS_LAST_PHONE_CHANGE_divide_OBS_60_CNT_SOCIAL_CIRCLE
DAYS_LAST_PHONE_CHANGE_divide_DEF_60_CNT_SOCIAL_CIRCLE
197 (356255, 329)


# Cat Features

In [38]:
cat_feats = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','NAME_TYPE_SUITE',
    'NAME_INCOME_TYPE','NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'FLAG_EMAIL', 'OCCUPATION_TYPE',
    'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY',
    'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
    'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
    'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14',
    'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19',
    'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21','WEEKDAY_APPR_PROCESS_START', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 
    'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'
]

#'FLAG_MOBIL' 'FLAG_EMP_PHONE' 'FLAG_WORK_PHONE' 'FLAG_CONT_MOBILE'\n 'FLAG_PHONE'
df.drop(cat_feats, inplace = True, axis = 1)
print(df.shape)

train = df[df['TARGET'].notnull()]
test = df[df['TARGET'].isnull()]
print(train.shape,test.shape)

test.drop('TARGET', axis = 1, inplace = True)
test.shape

(356255, 287)
(307511, 287) (48744, 287)


/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(48744, 286)

In [39]:
train_ext = pd.read_pickle(PATH+ 'inter/ext_linear.pkl')
# test_mean = pd.read_pickle(PATH + 'inter/test_mean.pkl')
test_ext = pd.DataFrame()

In [40]:
train = pd.concat([train, train_ext], axis = 1)

test = pd.concat([test, test_ext], axis = 1)

In [41]:
train.to_pickle(PATH + 'inter/06_22_train_mean.pkl')
test.to_pickle(PATH + 'inter/06_22_test_mean.pkl')

In [42]:
train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,DAYS_LAST_PHONE_CHANGE_divide_DAYS_REGISTRATION,DAYS_LAST_PHONE_CHANGE_divide_DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE_divide_CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE_divide_OBS_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE_divide_DEF_30_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE_divide_OBS_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE_divide_DEF_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_PRED_1,EXT_SOURCE_PRED_2,EXT_SOURCE_PRED_3
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0.310855,0.534906,-1134.0,-567.000000,-567.000000,-567.000000,-567.000000,0.184694,0.434859,0.417202
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0.698145,2.845361,-414.0,-828.000000,-inf,-828.000000,-inf,0.652030,0.603857,0.520355
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0.191315,0.322007,-815.0,-inf,-inf,-inf,-inf,0.550152,0.505275,0.542312
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0.062748,0.253180,-308.5,-308.500000,-inf,-308.500000,-inf,0.646242,0.502573,0.576674
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0.256553,0.319838,-1106.0,-inf,-inf,-inf,-inf,0.532783,0.496964,0.564171
